In [1]:
import pandas as pd
from prophet import Prophet
# from neuralprophet import NeuralProphet# as Prophet
# from neuralprophet import set_log_level
# set_log_level("ERROR")

from dotenv import load_dotenv
from pathlib import Path
import os

# from ezekial.prophet import Prophet
import yfinance as yf
import requests
import base64
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from multiprocessing import Pool, cpu_count, Process#, freeze_support
import time
import warnings
import sys

In [2]:
# Arguements
warnings.filterwarnings("ignore")
load_dotenv()
url = os.getenv("DJANGO_SERVER_URL")
weeeks_prior_data_end_date = int(sys.argv[1]) #260
forecast_days_ahead = int(sys.argv[2]) #90
ticker_path = sys.argv[3]#'data/S&P500 tickers.csv'
forecast_img_path = sys.argv[4]#'images/forecast_temp/forecast.png'

In [12]:
# # Arguements
# warnings.filterwarnings("ignore")
# load_dotenv()
# url = os.getenv("DJANGO_SERVER_URL")
# weeeks_prior_data_end_date = 12
# forecast_days_ahead = 90
# ticker_path = 'data/S&P500 tickers.csv'
# forecast_img_path = 'images/forecast_temp/forecast.png'

In [4]:
# print(weeeks_prior_data_end_date)
# print(forecast_days_ahead)
# print(ticker_path)
# print(forecast_img_path)

In [5]:
def load_tickers(ticker_path):
    """
    Takes five hundred tickers, which is max calls of yfinance for public ip.
    """
    ticker_file_path = Path(ticker_path)
    ticker_df = pd.read_csv(ticker_file_path)
    prophet_ticker_list = ticker_df['Symbol'].head(500).to_list()
    return prophet_ticker_list

def django_get(url):
    response = requests.get(url)
    response_json = response.json()
    return response_json

def clear_django_sql_db(url):
    """
    Clears the sql db in the django api.
    "DJANGO_SERVER_URL" is provided in .env file.
    """

    try:
        # load_dotenv()
        # url = os.getenv("DJANGO_SERVER_URL")

        django_api_items = django_get(url)

        if len(django_api_items) != 0:
            for x, index_id in enumerate(django_api_items):
                x += 1
                str_indx = str(index_id['id'])

                if x == 1:
                    url += str(str_indx)
                    requests.delete(url)
                    # print(f'entered if block\t{url}')
                else:
                    try:
                        url = url[:-len(str_indx)] + str_indx
                        requests.delete(url)
                        # print(f'entered else block\t{url}')
                    # For when the number of digits change
                    except:
                        url = url[:-len(str_indx) + 1] + str_indx
                        requests.delete(url)                    
                        # print(f'entered else block\t{url}')
            print(f"Django SQL DB cleared:\n\n{django_get(url)}")
        else:
            print(f"Django SQL DB Already empty:\n\n{django_get(url)}")
    except:
        print(f"Django server did not clear:\n\n{django_get(url)}")

def run_prophet(timeserie, forecast_ahead = forecast_days_ahead):
    model = Prophet(yearly_seasonality=False,daily_seasonality=False)
    model.fit(timeserie)
    forecast = model.make_future_dataframe(periods=forecast_ahead, include_history=True)
    forecast = model.predict(forecast)
    return forecast

# def make_plots(predictions, url = url):
# # def make_plots(predictions, series, adj_close_df, url = url):
#     # Attempt to clear the SQL DB if it fails initially before uploading the data again, 
#     # to reduce DB size for POST request on the APP side
#     try: clear_django_sql_db(url = url)
#     except: 
#         try: clear_django_sql_db(url = url)
#         except: error_msg = '\t##### Could not clear the Django SQL DB  #####'

#     for idx, fig_plot in enumerate(predictions):
#         forecast_plot = pd.merge(left=fig_plot, right=series[idx], on='ds', how='outer')
#         ticker_sym = adj_close_df.columns[idx]
#         forecast_plot = forecast_plot[-(forecast_days_ahead - 10) - 30:-(forecast_days_ahead - 10)]
#         forecast_plot.index = forecast_plot['ds']

#         with plt.style.context('dark_background'):
#             fig, ax = plt.subplots()
#             forecast_plot[['y','yhat']].plot(ax=ax, figsize = (8,10), style=['o-','--'],color=['r','lightblue'],linewidth=1)
#             ax.set_xlabel('Date', fontdict={'size':12})
#             ax.set_ylabel('Price', fontdict={'size':12})
#             ax.yaxis.set_major_formatter('${x:,.0f}')
#             ax.legend(['Actual','Forecasted'], title= f'{ticker_sym}', loc='upper center', frameon=False)
#             # ax.set_facecolor('xkcd:black')
#         # plt.show()
#         try:
#             fig.savefig(forecast_img_path)
#             with open(forecast_img_path, "rb") as image2string:
#                 converted_string = base64.b64encode(image2string.read())
#             image2string.close()

#             json_post = {
#                 "ticker": f"{ticker_sym}",
#                 "time": f'{today}'[:15],
#                 "encoded_string": f"{converted_string}"
#             }

#             try:
#                 requests.post(url,json_post)
#                 print(f"+++ {ticker_sym} success!")
#             except:
#                 print(f"\t--- {ticker_sym} failed ---")
#         except:
#             print(f"\t---# {ticker_sym} failed #---")
#             print("\n\n\tyfinance API daily limit reached.")

def make_plots(prediction, seri, adj_cl):

    forecast_plot = pd.merge(left=prediction, right=seri, on='ds', how='outer')
    ticker_sym = adj_cl
    
    # def weather_emoticons(i):
    """
    Returns the percent change for the first 10 forecasted values,
    taking the dot product of 1000 and casting it as an int.
    Outliers are automatically converted to max values of the
    ordinal scale, (-5, 5).
    """
    # forecast_plot = pd.merge(left=prediction, right=seri, on='ds', how='outer')
    forecast_plot.index = forecast_plot['ds']
    def correct_max_outliers (z):     
        if z > 5: z = 5
        elif z < -5: z = -5
        return z
    weather_df = forecast_plot[['yhat']].pct_change()[-forecast_days_ahead:-(forecast_days_ahead - 10)].dot([1000]).astype(int).apply(correct_max_outliers)
    # return weather_df
    weather_df = [[str(x)[-22:] for x in weather_df.index], [y for y in weather_df]]

    forecast_plot = forecast_plot[-(forecast_days_ahead - 10) - 30:-(forecast_days_ahead - 10)]
    # forecast_plot.index = forecast_plot['ds']

    with plt.style.context('dark_background'):
        fig, ax = plt.subplots()
        forecast_plot[['y','yhat']].plot(ax=ax, figsize = (8,10), style=['o-','--'],color=['r','lightblue'],linewidth=1)
        ax.set_xlabel('Date', fontdict={'size':12})
        ax.set_ylabel('Price', fontdict={'size':12})
        ax.yaxis.set_major_formatter('${x:,.0f}')
        ax.legend(['Actual','Forecasted'], title= f'{ticker_sym}', loc='upper center', frameon=False)
        # ax.set_facecolor('xkcd:black')
    # plt.show()
    try:
        # new_forecast_img_path = forecast_img_path + F'_{ticker_sym}.png'
        fig.savefig(forecast_img_path)
        with open(forecast_img_path, "rb") as image2string:
            converted_string = base64.b64encode(image2string.read())
        image2string.close()

        json_post = {
            "ticker": f"{ticker_sym}",
            "time": f'{today}'[:16],
            "encoded_string": f"{converted_string}",
            "emoticons": f"{weather_df}"
        }

        try:
            requests.post(url,json_post)
            print(f"+++ {ticker_sym} success!")
        except:
            print(f"\t--- {ticker_sym} failed ---")
    except:
        print(f"\t---# {ticker_sym} failed #---")
        print("\n\n\tyfinance API daily limit reached.")

In [6]:
# weather_df = forecast_plot[['yhat']].pct_change()[-forecast_days_ahead:-(forecast_days_ahead - 10)].dot([1000]).astype(int).apply(correct_max_outliers)

# this_index = [[str(x)[-22:] for x in weather_df.index], [y for y in weather_df]]
# this_index

In [7]:
# start_time = time.time()
# today = datetime.today()
# start_date = today - timedelta(weeks=weeeks_prior_data_end_date)
# start_date = start_date.replace(second=0, microsecond=0)

# data = yf.download(load_tickers(ticker_path), start=start_date)
# adj_close_df = data['Adj Close']
# adj_close_df = adj_close_df.dropna(axis=1, how='all')
# series = [adj_close_df[f'{col}'].reset_index().rename({'Date':'ds', f'{col}':'y'}, axis=1) for col in adj_close_df]

# start_time0 = time.time()
# p = Pool(cpu_count())
# predictions = list(tqdm(p.imap(run_prophet, series), total=len(series)))
# p.close()
# p.join()
# print("--- %s seconds ---" % (time.time() - start_time0))

# # make_plots(url)

# # try: print(error_msg)
# # except: print("\t+++++ The SQL DB was successfully cleared +++++")

# # print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
today = datetime.today()
start_date = today - timedelta(weeks=weeeks_prior_data_end_date)
start_date = start_date.replace(second=0, microsecond=0)

data = yf.download(load_tickers(ticker_path), start=start_date)
adj_close_df = data['Adj Close']
adj_close_df = adj_close_df.dropna(axis=1, how='all')
series = [adj_close_df[f'{col}'].reset_index().rename({'Date':'ds', f'{col}':'y'}, axis=1) for col in adj_close_df]


start_time00 = time.time()
p1 = Pool(cpu_count())
predictions1 = list(tqdm(p1.imap(run_prophet, series), total=len(series)))
p1.close()
p1.join()
print("--- %s seconds ---" % (time.time() - start_time00))

[*********************100%***********************]  500 of 500 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


  0%|          | 0/498 [00:00<?, ?it/s]


Initial log joint probability = -23.401
Iteration  1. Log joint probability =    74.8325. Improved by 98.2335.

Initial log joint probability = -23.5864

Initial log joint probability = -23.3151
Iteration  2. Log joint probability =    129.271. Improved by 54.4386.
Iteration  1. Log joint probability =    41.5589. Improved by 65.1453.
Iteration  3. Log joint probability =    157.179. Improved by 27.9079.
Iteration  1. Log joint probability =    99.0951. Improved by 122.41.
Iteration  2. Log joint probability =    109.865. Improved by 68.3061.
Iteration  4. Log joint probability =    170.383. Improved by 13.2037.
Iteration  2. Log joint probability =     134.82. Improved by 35.7252.
Iteration  3. Log joint probability =     125.03. Improved by 15.1646.


Initial log joint probability = -23.3165
Initial log joint probability = -23.3387
Iteration  3. Log joint probability =     213.81. Improved by 78.9894.
Iteration  5. Log joint probability =    170.487. Improved by 0.104637.
Iteration 

In [13]:
# Attempt to clear the SQL DB if it fails initially before uploading the data again, 
# to reduce DB size for POST request on the APP side
try: clear_django_sql_db(url = url)
except: 
    try: clear_django_sql_db(url = url)
    except: error_msg = '\t##### Could not clear the Django SQL DB  #####'

mapping_star = [(predictions1[i], series[i], adj_close_df.columns[i]) for i in range(len(predictions1))]

start_time1 = time.time()

p = Pool(cpu_count())
p.starmap(make_plots, mapping_star)
p.close()
p.join()
print("--- %s seconds ---" % (time.time() - start_time1))

Django SQL DB Already empty:

[]
+++ A success!
+++ CRL success!
+++ AES success!
+++ AAL success!
+++ CRM success!
+++ AAP success!
+++ CL success!
+++ AAPL success!
+++ AVGO success!
+++ ABBV success!
+++ CSCO success!
+++ BKR success!
+++ ABC success!
+++ AVY success!
+++ ABMD success!
+++ BLK success!
+++ ABT success!
+++ CSX success!
+++ BLL success!
+++ AMP success!
+++ ACN success!
+++ AFL success!
+++ CLX success!
+++ CBRE success!
+++ CMA success!
+++ AWK success!
+++ CMCSA success!
+++ AIG success!
+++ CTAS success!
+++ AMT success!
+++ CTLT success!
+++ AMZN success!
+++ AIZ success!
+++ ANET success!
+++ AXP success!
+++ CTRA success!
+++ AJG success!
+++ ADBE success!
+++ CTSH success!
+++ ADI success!
+++ AZO success!
+++ CTVA success!
+++ ADM success!
+++ CME success!
+++ CCI success!
+++ CTXS success!
+++ AKAM success!
+++ CCL success!
+++ CVS success!
+++ CDAY success!
+++ ALB success!
+++ CDNS success!
+++ BMY success!
+++ ALGN success!
+++ BR success!
+++ CDW success

In [14]:
try: print(error_msg)
except: print("\t+++++ The SQL DB was successfully cleared +++++")

print("--- %s seconds ---" % (time.time() - start_time))

	+++++ The SQL DB was successfully cleared +++++
--- 1189.657098531723 seconds ---


In [10]:
# p.map(make_plots, predictions)

In [11]:
# clear_django_sql_db(url)

In [12]:
# api_data = django_get(url)#[0]['id']

In [13]:
# len(api_data)

In [14]:
# class save_ticker_image:
#     def __init__(self, url, img_path, ticker):
#         self.url = url
#         self.img_path = img_path
#         self.ticker = ticker
    
#     def django_get(self):
#         response = requests.get(self.url)
#         response_json = response.json()
#         return response_json

#     def select_ticker(self):
#         django_api_dict = self.django_get()
#         for dictionary in django_api_dict:   
#             try:
#                 if dictionary['ticker'] == self.ticker:
#                     encoded_plot = dictionary['encoded_string']
#             except: return print("Ticker Not Found in Forecast Database")
#         # Decode string and save as a .png
#         try:
#             decodeit = open(self.img_path, 'wb')
#             decodeit.write(base64.b64decode(encoded_plot[2:].encode()))
#             decodeit.close()
#             print("Image Saved Successfully")
#         except: return print("Could Not Save Plot Ticker Not Found")

In [15]:
# forecast_decoded_img_path = Path('images/forecast_temp/decoded.png')
# save_ticker_image(url=url, img_path=forecast_decoded_img_path, ticker='AAPL').select_ticker()

In [143]:
# def weather_emoticons(i):
#     """
#     Returns the percent change for the first 10 forecasted values,
#     taking the dot product of 1000 and casting it as an int.
#     Outliers are automatically converted to max values of the
#     ordinal scale, (-5, 5).
#     """
#     forecast_plot = pd.merge(left=mapping_star[i][0], right=mapping_star[i][1], on='ds', how='outer')
#     forecast_plot.index = forecast_plot['ds']
#     def correct_max_outliers (z):     
#         if z > 5: z = 5
#         elif z < -5: z = -5
#         return z
#     weather_df = forecast_plot[['yhat']].pct_change()[-forecast_days_ahead:-(forecast_days_ahead - 10)].dot([1000]).astype(int).apply(correct_max_outliers)
#     return weather_df

In [118]:
# def correct_max_outliers (z):     
#     if z > 5: z = 5
#     elif z < -5: z = -5
#     return z
    
# this_list = pd.Series([6, 5, 3, 0, 100, -100, -5, -3, -2])
# # this_lamb(this_list)

# this_list.map(this_lamb)

In [145]:
# weather_emoticons(1)

In [129]:
# weather_emoticons(2)

In [130]:
# weather_emoticons(3)

In [125]:
# weather_emoticons(4)

In [126]:
# weather_emoticons(5)

In [127]:
# weather_emoticons(6)

In [196]:
# start_time = time.time()
# today = datetime.today()
# start_date = today - timedelta(weeks=weeeks_prior_data_end_date)
# start_date = start_date.replace(second=0, microsecond=0)

# data = yf.download('AAPL', start=start_date)

[*********************100%***********************]  1 of 1 completed


In [197]:
# data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

## Sentiment Analysis API Exploratory Analysis

In [41]:
# import requests
# import json

# load_dotenv()
# YOUR_API_TOKEN = os.getenv("MARKETAUX_TOKEN")
# url = f'https://api.marketaux.com/v1/news/all?symbols=TSLA,AMZN,MSFT&filter_entities=true&group_similar=false&language=en&api_token={YOUR_API_TOKEN}'

# setntiment_data = requests.get(url).json()

In [44]:
# len(setntiment_data['data'])

In [57]:
# setntiment_data['data'][0]

{'uuid': 'bd6dc326-b53a-4b53-8b1c-57383127111f',
 'title': 'Does the Betashares Nasdaq 100 ETF pay dividends?',
 'description': 'The BetaShares Nasdaq 100 ETF (ASX: NDQ) is a popular ASX choice for its US tech share exposure. But does this ETF pay dividend distributions?',
 'keywords': '',
 'snippet': 'This exchange-traded fund gives investors exposure to much of the US tech sector.\n\nFor an ETF to pay dividend distributions, it usually needs to hold dividend s...',
 'url': 'https://www.fool.com.au/2022/04/09/does-the-betashares-nasdaq-100-etf-pay-dividends/',
 'image_url': 'https://www.fool.com.au/wp-content/uploads/2022/02/dividend-13-16.9.jpg',
 'language': 'en',
 'published_at': '2022-04-08T23:00:00.000000Z',
 'source': 'fool.com.au',
 'relevance_score': None,
 'entities': [{'symbol': 'AMZN',
   'name': 'Amazon.com, Inc.',
   'exchange': 'NASDAQ',
   'exchange_long': 'NASDAQ Stock Exchange',
   'country': 'us',
   'type': 'equity',
   'industry': 'Consumer Cyclical',
   'match_sco

In [48]:
# setntiment_data['data'][0].keys()

dict_keys(['uuid', 'title', 'description', 'keywords', 'snippet', 'url', 'image_url', 'language', 'published_at', 'source', 'relevance_score', 'entities', 'similar'])

In [59]:
# setntiment_data['data'][0]['entities'][0].keys()

dict_keys(['symbol', 'name', 'exchange', 'exchange_long', 'country', 'type', 'industry', 'match_score', 'sentiment_score', 'highlights'])

In [90]:
# setntiment_data['data'][0]['entities'][0]['symbol']

'AMZN'

In [55]:
# setntiment_data['data'][0]['entities'][0]['sentiment_score']

0

In [70]:
# setntiment_data['data'][0]['entities'][1].keys()

dict_keys(['symbol', 'name', 'exchange', 'exchange_long', 'country', 'type', 'industry', 'match_score', 'sentiment_score', 'highlights'])

In [67]:
# setntiment_data['data'][0]['entities'][1]

{'symbol': 'MSFT',
 'name': 'Microsoft Corporation',
 'exchange': 'NASDAQ',
 'exchange_long': 'NASDAQ Stock Exchange',
 'country': 'us',
 'type': 'equity',
 'industry': 'Technology',
 'match_score': 56.686882,
 'sentiment_score': 0.4201,
 'highlights': [{'highlight': 'You’ll find everything from Apple Inc (NASDAQ: AAPL), <em>Microsoft</em> <em>Corporation</em> (<em>NASDAQ</em>: <em>MSFT</em>) and Amazon.com Inc (NASDAQ: AMZN) to Alphabet Inc (NASDAQ: GOOG)(NASDAQ: GOOGL), Tesla Inc (NASDAQ: TSLA) and Netflix Inc (NASDAQ: NFLX) on the Nasdaq.',
   'sentiment': 0,
   'highlighted_in': 'main_text'},
  {'highlight': 'Apple and <em>Microsoft</em> both dole out quarterly dividends. NVIDIA Corporation (NASDAQ: NVDA), another top holding in NDQ’s portfolio, is also a dividend share. As are Costco and Intel. PepsiCo. Yes, the company behind Pepsi-Cola is also a Nasdaq share and has been paying a dividend that has increased every year for the past 49 years.',
   'sentiment': 0.8402,
   'highligh

In [71]:
# setntiment_data['data'][0]['entities'][1]['symbol']

'MSFT'

In [79]:
# setntiment_data['data'][0]['entities'][1]['highlights'][0]['sentiment']

0

In [80]:
# setntiment_data['data'][0]['entities'][1]['highlights'][1]['sentiment']

0.8402

In [83]:
# setntiment_data['data'][0]['entities'][2].keys()

dict_keys(['symbol', 'name', 'exchange', 'exchange_long', 'country', 'type', 'industry', 'match_score', 'sentiment_score', 'highlights'])

In [82]:
# setntiment_data['data'][0]['entities'][2]

{'symbol': 'TSLA',
 'name': 'Tesla, Inc.',
 'exchange': 'NASDAQ',
 'exchange_long': 'NASDAQ Stock Exchange',
 'country': 'us',
 'type': 'equity',
 'industry': 'Consumer Cyclical',
 'match_score': 58.3895,
 'sentiment_score': 0.38585,
 'highlights': [{'highlight': 'You’ll find everything from Apple Inc (NASDAQ: AAPL), Microsoft Corporation (NASDAQ: MSFT) and Amazon.com Inc (NASDAQ: AMZN) to Alphabet Inc (NASDAQ: GOOG)(NASDAQ: GOOGL), <em>Tesla</em> <em>Inc</em> (<em>NASDAQ</em>: <em>TSLA</em>) and Netflix Inc (NASDAQ: NFLX) on the Nasdaq.',
   'sentiment': 0,
   'highlighted_in': 'main_text'},
  {'highlight': 'Many US tech shares, including Amazon, <em>Tesla</em>, Netflix, and Alphabet, have never paid a dividend.\n\nHowever, quite a few of NDQ’s top holdings are dividend payers. Apple and Microsoft both dole out quarterly dividends. NVIDIA Corporation (NASDAQ: NVDA), another top holding in NDQ’s portfolio, is also a dividend share. As are Costco and Intel.',
   'sentiment': 0.7717,
   

In [84]:
# setntiment_data['data'][0]['entities'][2]['symbol']

'TSLA'

In [88]:
# setntiment_data['data'][0]['entities'][2]['sentiment_score']

0.38585

In [89]:
# setntiment_data['data'][0]['entities'][2]['highlights'][1]['sentiment']

0.7717

In [68]:
# setntiment_data['data'][0]['entities'][2]

{'symbol': 'TSLA',
 'name': 'Tesla, Inc.',
 'exchange': 'NASDAQ',
 'exchange_long': 'NASDAQ Stock Exchange',
 'country': 'us',
 'type': 'equity',
 'industry': 'Consumer Cyclical',
 'match_score': 58.3895,
 'sentiment_score': 0.38585,
 'highlights': [{'highlight': 'You’ll find everything from Apple Inc (NASDAQ: AAPL), Microsoft Corporation (NASDAQ: MSFT) and Amazon.com Inc (NASDAQ: AMZN) to Alphabet Inc (NASDAQ: GOOG)(NASDAQ: GOOGL), <em>Tesla</em> <em>Inc</em> (<em>NASDAQ</em>: <em>TSLA</em>) and Netflix Inc (NASDAQ: NFLX) on the Nasdaq.',
   'sentiment': 0,
   'highlighted_in': 'main_text'},
  {'highlight': 'Many US tech shares, including Amazon, <em>Tesla</em>, Netflix, and Alphabet, have never paid a dividend.\n\nHowever, quite a few of NDQ’s top holdings are dividend payers. Apple and Microsoft both dole out quarterly dividends. NVIDIA Corporation (NASDAQ: NVDA), another top holding in NDQ’s portfolio, is also a dividend share. As are Costco and Intel.',
   'sentiment': 0.7717,
   

In [60]:
# setntiment_data['data'][0]['entities'][0]['symbol']

'AMZN'

In [61]:
# setntiment_data['data'][0]['entities'][0]['sentiment_score']

0

In [66]:
# setntiment_data['data'][0]['entities'][0]['highlights'][0].keys()

dict_keys(['highlight', 'sentiment', 'highlighted_in'])

In [46]:
# setntiment_data['data'][1]

{'uuid': 'edde8471-852a-472b-ba6b-a756ab1334a4',
 'title': 'Amazon calls for election re-run after workers voted for first U.S. union By Reuters',
 'description': 'Amazon calls for election re-run after workers voted for first U.S. union',
 'keywords': '',
 'snippet': '© Reuters. FILE PHOTO: Amazon Labour Union (ALU) organiser Christian Smalls reacts as ALU members celebrate official victory after hearing results regarding th...',
 'url': 'https://www.investing.com/news/stock-market-news/amazon-calls-for-election-rerun-after-workers-voted-for-first-us-union-2800944',
 'image_url': 'https://i-invdn-com.investing.com/news/LYNXNPEC1412T_L.jpg',
 'language': 'en',
 'published_at': '2022-04-08T22:55:49.000000Z',
 'source': 'investing.com',
 'relevance_score': None,
 'entities': [{'symbol': 'AMZN',
   'name': 'Amazon.com, Inc.',
   'exchange': 'NASDAQ',
   'exchange_long': 'NASDAQ Stock Exchange',
   'country': 'us',
   'type': 'equity',
   'industry': 'Consumer Cyclical',
   'match_score': 

In [47]:
# setntiment_data['data'][2]

{'uuid': '28722cb8-5688-4ca2-b11b-9803a4d41a71',
 'title': 'Amazon calls for election re-run after workers voted for first U.S. union',
 'description': 'NEW YORK — Amazon.com Inc on Friday called for an election re-run after workers at a New York City warehouse voted to create the company’s first U.S. union,…',
 'keywords': '',
 'snippet': 'This advertisement has not loaded yet, but your article continues below.\n\nAmazon calls for election re-run after workers voted for first U.S. union\n\nArticle con...',
 'url': 'https://financialpost.com/pmn/business-pmn/amazon-calls-for-election-re-run-after-workers-voted-for-first-u-s-union-2',
 'image_url': 'https://storage.googleapis.com/pmd-stage-northamerica-northeast1-dcs-static-files/10.2.2/websites/images/fp/favicon-fp.ico',
 'language': 'en',
 'published_at': '2022-04-08T22:50:08.000000Z',
 'source': 'financialpost.com',
 'relevance_score': None,
 'entities': [{'symbol': 'AMZN',
   'name': 'Amazon.com, Inc.',
   'exchange': 'NASDAQ',
   '